---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [31]:
income    =  75000                     # taxable income
status    = 'single'                   # filing status
# choiceS: 'single', 'married filing jointly', 'married filing separately', 'head of household'

# TAX BRACKETS

In [32]:
# 2023 information
BRACKETS = {
    'single':                   [0, 11000, 44725,  95375, 182100, 231250, 578125],
    'married filing jointly':   [0, 22000, 89450, 190750, 364200, 462500, 693750],
    'married filing separately':[0, 11000, 44725,  95375, 182100, 231250, 346875],
    'head of household':        [0, 15700, 59850,  95350, 182100, 231250, 578100]
}

RATES  = [0.1, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]

# CALCULATIONS

In [33]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

def tax(income, status):
    LIMITS = BRACKETS[status]
    tax = 0
    for i, limit in enumerate(LIMITS):
        if i < len(LIMITS)-1:
            upper = LIMITS[i+1]
        else:
            upper = np.inf
        if income >= limit:
            tax +=  RATES[i] * max(0, min(income-limit, upper - limit))
    return tax


tax_due = tax(income,status)
print(f'For filing status {status} with taxable income of ${income:,.0f}:')
print(f'\ttotal tax:\t${tax_due:,.0f}')

# Effective and marginal rates
def marginal(income, status):
    LIMITS = BRACKETS[status]
    for i, limit in enumerate(LIMITS):
        if income >= limit:
            rate = RATES[i]
    return rate

t_marginal  = marginal(income, status)
t_effective = tax_due / income
print(f'\teffective rate:\t{t_effective:.1%}')
print(f'\tmarginal rate:\t{t_marginal:.1%}')



For filing status single with taxable income of $75,000:
	total tax:	$11,808
	effective rate:	15.7%
	marginal rate:	22.0%


# FIGURE

In [34]:
import plotly.graph_objects as go

incomes = np.arange(1000,1001000,1000)
marginal_pcts  = [marginal(x,status) for x in incomes]
effective_pcts = [tax(x,status)/x for x in incomes]

# Scatter plot of average tax rate
fig = go.Figure()
trace  = go.Scatter(x=incomes, y=effective_pcts, mode="lines",name='Effective')
fig.add_trace(trace)

trace1  = go.Scatter(x=incomes, y=marginal_pcts, mode="lines", name='Marginal')
fig.add_trace(trace1)

# Formatting
fig.update_layout(
    hovermode='x unified',
    xaxis_title='Taxable Income',
    xaxis_tickformat=",.0f",
    xaxis_tickprefix='$',
    yaxis_title='Tax Rate',
    yaxis_tickformat=".1%",
    title='Filing Status: ' + status.title(),
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01)
)
fig.show()
